In [11]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts.prompt import PromptTemplate

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)


examples = [
    {
        "question": "What do you know about the movie 'Big Fish'",
        "answer": """
        Here is what I know:
        Director: Tim Burton
        Cast: Ewan McGregor, Alison Lohman
        Budget: $70,000,000
        Box Office Profits: $122,919,055
        Genre: Drama, Fantasy, Romance
        Synopsis: The film explores Edward's life through the tall tales he tells, filled with fantastical characters and incredible adventures. As Edward nears the end of his life, Will, who has always been skeptical of his father's stories, tries to separate fact from fiction to understand who his father truly is. The movie blends reality and fantasy, ultimately revealing how stories can shape our identities and connections with others.
        """,
    },
    {
        "question": "What do you know about the movie 'The Rocky Horror Picture Show'",
        "answer": """
        Here is what I know:
        Director: Jim Sharman 
        Cast: Tim Curry, Susan Sarandon
        Budget: $1,400,000
        Box Office Profits: $170,000,000
        Genre: Musical, Horror, Comedy, SF
        Synopsis: 
        """,
    },
    {
        "question": "What do you know about the movie 'The Fall'",
        "answer": """
        Here is what I know:
        Director: Tarsem Singh
        Cast: Lee Pace, Catinca Untaru
        Budget: $30,000,000
        Box Office Profits: $3,669,465
        Genre: Adventure, Fanasy, Drama
        Synopsis: The story is set in a Los Angeles hospital in the 1920s, where a bedridden stuntman named Roy Walker befriends a young immigrant girl named Alexandria. To pass the time, Roy begins telling Alexandria an epic story about five mythical heroes on a quest to defeat an evil ruler. As the line between reality and fantasy blurs, Roy's own despair seeps into the tale, leading both him and Alexandria on a journey of hope, redemption, and imagination. The film is renowned for its breathtaking cinematography and unique storytelling approach.
        """,
    },
]


example_prompt = PromptTemplate.from_template("Human: {question}\nAI:{answer}")

prompt = FewShotPromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
    suffix="Human: What do you know about {movie}?",
    input_variables=["movie"],
)

chain = prompt | chat

chain.invoke({"movie": "The Lord of the Rings"})

AI: 
        Here is what I know:
        Director: Peter Jackson
        Cast: Elijah Wood, Ian McKellen, Viggo Mortensen
        Budget: $281,000,000 (for the entire trilogy)
        Box Office Profits: $2,981,000,000 (for the entire trilogy)
        Genre: Fantasy, Adventure
        Synopsis: Based on the novels by J.R.R. Tolkien, The Lord of the Rings follows the journey of Frodo Baggins as he sets out to destroy the One Ring and defeat the dark lord Sauron. Along the way, he is joined by a fellowship of diverse characters who must overcome various challenges and obstacles in order to save Middle-earth from the forces of evil. The trilogy is known for its epic scale, intricate world-building, and memorable characters.

AIMessageChunk(content='AI: \n        Here is what I know:\n        Director: Peter Jackson\n        Cast: Elijah Wood, Ian McKellen, Viggo Mortensen\n        Budget: $281,000,000 (for the entire trilogy)\n        Box Office Profits: $2,981,000,000 (for the entire trilogy)\n        Genre: Fantasy, Adventure\n        Synopsis: Based on the novels by J.R.R. Tolkien, The Lord of the Rings follows the journey of Frodo Baggins as he sets out to destroy the One Ring and defeat the dark lord Sauron. Along the way, he is joined by a fellowship of diverse characters who must overcome various challenges and obstacles in order to save Middle-earth from the forces of evil. The trilogy is known for its epic scale, intricate world-building, and memorable characters.')